In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum

# ---------------------------
# Step 1: Load Data
# ---------------------------
# df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
# df_truck = pd.read_csv("round1-day2_truck_node_data.csv")
# df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
# df_problem = pd.read_csv("round1-day2_problem_data.csv")

# Load the CSV files
df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
df_truck = pd.read_csv("round1-day2_truck_node_data (1).csv")
df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
df_problem = pd.read_csv("round1-day2_problem_data.csv")

print("Demand Node Columns:", df_demand.columns.tolist())
print("Truck Node Columns:", df_truck.columns.tolist())
print("Demand-Truck Columns:", df_demand_truck.columns.tolist())
print("Problem Data Columns:", df_problem.columns.tolist())

# ---------------------------
# Step 2: Build Data Structures
# ---------------------------

# 2.1: Truck list
truck_locations = df_truck['index'].tolist()

# 2.2: Demand node list
#    We'll also keep a set for quick membership checks
demand_nodes = df_demand['index'].tolist()
demand_set = set(demand_nodes)

# 2.3: Build a dictionary for scaled_demand from demand-truck data
#    We only record pairs (i, j) where scaled_demand > 0, to form "validPairs".
scaled_demand = {}
valid_pairs = []
for _, row in df_demand_truck.iterrows():
    i = row['truck_node_index']
    j = row['demand_node_index']
    sd = row['scaled_demand']  # how many burritos truck i can serve to node j
    if sd > 0:
        scaled_demand[(i, j)] = sd
        valid_pairs.append((i, j))

# 2.4: Problem data for profit
burrito_price = df_problem['burrito_price'].iloc[0]       # e.g. 10
ingredient_cost = df_problem['ingredient_cost'].iloc[0]   # e.g. 5
profit_per_burrito = burrito_price - ingredient_cost       # e.g. 5

# 2.5: Fixed cost per truck
truck_cost_value = df_problem['truck_cost'].iloc[0]  # e.g. 250
truck_fixed_cost = { t: truck_cost_value for t in truck_locations }

# ---------------------------
# Step 3: Create Gurobi Model
# ---------------------------
m = gp.Model("Burrito_Profit_UsingScaledDemand")

# 3.1: Decision Variables
# z_i: 1 if truck i is open
z = m.addVars(truck_locations, vtype=GRB.BINARY, name="OpenTruck")

# x_{i,j}: 1 if truck i serves node j (which means scaled_demand[(i,j)] burritos)
# Only define x_{i,j} for valid pairs
x = {}
for (i, j) in valid_pairs:
    x[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f"Assign_{i}_{j}")

# 3.2: Constraints

# A) Each demand node must be served exactly once
#    Summation over all trucks i for which (i, j) is a valid pair
for j in demand_nodes:
    # Find which trucks can serve j (scaled_demand>0)
    trucks_that_can_serve_j = [(i, jj) for (i, jj) in valid_pairs if jj == j]
    # If no truck can serve j, the model might be infeasible. 
    if trucks_that_can_serve_j:
        m.addConstr(
            quicksum(x[(i, jj)] for (i, jj) in trucks_that_can_serve_j) == 1,
            name=f"OneTruck_{j}"
        )
    else:
        # Potentially raise an error or handle if no truck can serve j
        print(f"WARNING: No valid truck can serve demand node {j}")

# B) Truck must be open if it serves any demand node
for (i, j) in valid_pairs:
    m.addConstr(x[(i, j)] <= z[i], name=f"Activation_{i}_{j}")

# (Optional) If you do have a capacity limit for each truck, define it:
# For now, let's assume no capacity limit or a big one. 
# If you have capacity[i], do:
# for i in truck_locations:
#     m.addConstr(
#         quicksum(scaled_demand[(i, j)] * x[(i, j)] for (ii, j) in valid_pairs if ii == i) <= capacity[i] * z[i],
#         name=f"Cap_{i}"
#     )

# 3.3: Objective Function
# Maximize total profit = sum( profit_per_burrito * scaled_demand[(i,j)] ) - sum( truck_cost * z[i] )
m.setObjective(
    quicksum( profit_per_burrito * scaled_demand[(i, j)] * x[(i, j)]
              for (i, j) in valid_pairs )
    - quicksum( truck_fixed_cost[i] * z[i] for i in truck_locations ),
    GRB.MAXIMIZE
)

# ---------------------------
# Step 4: Optimize
# ---------------------------
m.optimize()

# ---------------------------
# Step 5: Print Results
# ---------------------------
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal Profit = {m.objVal:.2f}\n")
    chosen_trucks = [i for i in truck_locations if z[i].X > 0.5]
    print("Chosen Truck Locations:", chosen_trucks)

    # For each chosen truck, see which demand nodes it serves
    for i in chosen_trucks:
        assigned_nodes = []
        total_burritos = 0
        for j in demand_nodes:
            if (i, j) in valid_pairs and x[(i, j)].X > 0.5:
                assigned_nodes.append(j)
                total_burritos += scaled_demand[(i, j)]
        if assigned_nodes:
            print(f"  Truck {i} serves: {assigned_nodes}, total scaled burritos = {total_burritos}")
else:
    print("No optimal solution found. Status =", m.status)


Demand Node Columns: ['index', 'name', 'x', 'y', 'demand']
Truck Node Columns: ['index', 'x', 'y']
Demand-Truck Columns: ['demand_node_index', 'truck_node_index', 'distance', 'scaled_demand']
Problem Data Columns: ['burrito_price', 'ingredient_cost', 'truck_cost']
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 405 rows, 395 columns and 1110 nonzeros
Model fingerprint: 0x98d73935
Variable types: 0 continuous, 395 integer (395 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -385.0000000
Presolve time: 0.00s
Presolved: 405 rows, 395 columns, 1110 nonz

In [ ]:
#  case two increase in ingredient cost thus cost of ingredient becomes $7 . 

In [7]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum

# ---------------------------
# Step 1: Load Data
# ---------------------------
# df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
# df_truck = pd.read_csv("round1-day2_truck_node_data.csv")
# df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
# df_problem = pd.read_csv("round1-day2_problem_data.csv")

# Load the CSV files
df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
df_truck = pd.read_csv("round1-day2_truck_node_data (1).csv")
df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
df_problem = pd.read_csv("round1-day2_problem_data.csv")

print("Demand Node Columns:", df_demand.columns.tolist())
print("Truck Node Columns:", df_truck.columns.tolist())
print("Demand-Truck Columns:", df_demand_truck.columns.tolist())
print("Problem Data Columns:", df_problem.columns.tolist())

# ---------------------------
# Step 2: Build Data Structures
# ---------------------------

# 2.1: Truck list
truck_locations = df_truck['index'].tolist()

# 2.2: Demand node list
#    We'll also keep a set for quick membership checks
demand_nodes = df_demand['index'].tolist()
demand_set = set(demand_nodes)

# 2.3: Build a dictionary for scaled_demand from demand-truck data
#    We only record pairs (i, j) where scaled_demand > 0, to form "validPairs".
scaled_demand = {}
valid_pairs = []
for _, row in df_demand_truck.iterrows():
    i = row['truck_node_index']
    j = row['demand_node_index']
    sd = row['scaled_demand']  # how many burritos truck i can serve to node j
    if sd > 0:
        scaled_demand[(i, j)] = sd
        valid_pairs.append((i, j))

# 2.4: Problem data for profit
burrito_price = df_problem['burrito_price'].iloc[0]       # e.g. 10
ingredient_cost = df_problem['ingredient_cost'].iloc[0]   # e.g. 5
ingredient_cost = 7
profit_per_burrito = burrito_price - ingredient_cost       # e.g. 5

# 2.5: Fixed cost per truck
truck_cost_value = df_problem['truck_cost'].iloc[0]  # e.g. 250
truck_fixed_cost = { t: truck_cost_value for t in truck_locations }

# ---------------------------
# Step 3: Create Gurobi Model
# ---------------------------
m = gp.Model("Burrito_Profit_UsingScaledDemand")

# 3.1: Decision Variables
# z_i: 1 if truck i is open
z = m.addVars(truck_locations, vtype=GRB.BINARY, name="OpenTruck")

# x_{i,j}: 1 if truck i serves node j (which means scaled_demand[(i,j)] burritos)
# Only define x_{i,j} for valid pairs
x = {}
for (i, j) in valid_pairs:
    x[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f"Assign_{i}_{j}")

# 3.2: Constraints

# A) Each demand node must be served exactly once
#    Summation over all trucks i for which (i, j) is a valid pair
for j in demand_nodes:
    # Find which trucks can serve j (scaled_demand>0)
    trucks_that_can_serve_j = [(i, jj) for (i, jj) in valid_pairs if jj == j]
    # If no truck can serve j, the model might be infeasible. 
    if trucks_that_can_serve_j:
        m.addConstr(
            quicksum(x[(i, jj)] for (i, jj) in trucks_that_can_serve_j) == 1,
            name=f"OneTruck_{j}"
        )
    else:
        # Potentially raise an error or handle if no truck can serve j
        print(f"WARNING: No valid truck can serve demand node {j}")

# B) Truck must be open if it serves any demand node
for (i, j) in valid_pairs:
    m.addConstr(x[(i, j)] <= z[i], name=f"Activation_{i}_{j}")

# (Optional) If you do have a capacity limit for each truck, define it:
# For now, let's assume no capacity limit or a big one. 
# If you have capacity[i], do:
# for i in truck_locations:
#     m.addConstr(
#         quicksum(scaled_demand[(i, j)] * x[(i, j)] for (ii, j) in valid_pairs if ii == i) <= capacity[i] * z[i],
#         name=f"Cap_{i}"
#     )

# 3.3: Objective Function
# Maximize total profit = sum( profit_per_burrito * scaled_demand[(i,j)] ) - sum( truck_cost * z[i] )
m.setObjective(
    quicksum( profit_per_burrito * scaled_demand[(i, j)] * x[(i, j)]
              for (i, j) in valid_pairs )
    - quicksum( truck_fixed_cost[i] * z[i] for i in truck_locations ),
    GRB.MAXIMIZE
)

# ---------------------------
# Step 4: Optimize
# ---------------------------
m.optimize()

# ---------------------------
# Step 5: Print Results
# ---------------------------
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal Profit = {m.objVal:.2f}\n")
    chosen_trucks = [i for i in truck_locations if z[i].X > 0.5]
    print("Chosen Truck Locations:", chosen_trucks)

    # For each chosen truck, see which demand nodes it serves
    for i in chosen_trucks:
        assigned_nodes = []
        total_burritos = 0
        for j in demand_nodes:
            if (i, j) in valid_pairs and x[(i, j)].X > 0.5:
                assigned_nodes.append(j)
                total_burritos += scaled_demand[(i, j)]
        if assigned_nodes:
            print(f"  Truck {i} serves: {assigned_nodes}, total scaled burritos = {total_burritos}")
else:
    print("No optimal solution found. Status =", m.status)


Demand Node Columns: ['index', 'name', 'x', 'y', 'demand']
Truck Node Columns: ['index', 'x', 'y']
Demand-Truck Columns: ['demand_node_index', 'truck_node_index', 'distance', 'scaled_demand']
Problem Data Columns: ['burrito_price', 'ingredient_cost', 'truck_cost']
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 405 rows, 395 columns and 1110 nonzeros
Model fingerprint: 0x2908da5c
Variable types: 0 continuous, 395 integer (395 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -1931.000000
Presolve time: 0.00s
Presolved: 405 rows, 395 columns, 1110 nonzeros
Variable types: 0 continuous, 395 integer (395 binary)

Root rela

In [3]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum

# ---------------------------
# Step 1: Load Data
# ---------------------------
# df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
# df_truck = pd.read_csv("round1-day2_truck_node_data.csv")
# df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
# df_problem = pd.read_csv("round1-day2_problem_data.csv")

# Load the CSV files
df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
df_truck = pd.read_csv("round1-day2_truck_node_data (1).csv")
df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
df_problem = pd.read_csv("round1-day2_problem_data.csv")

print("Demand Node Columns:", df_demand.columns.tolist())
print("Truck Node Columns:", df_truck.columns.tolist())
print("Demand-Truck Columns:", df_demand_truck.columns.tolist())
print("Problem Data Columns:", df_problem.columns.tolist())

# ---------------------------
# Step 2: Build Data Structures
# ---------------------------

# 2.1: Truck list
truck_locations = df_truck['index'].tolist()

# 2.2: Demand node list
#    We'll also keep a set for quick membership checks
demand_nodes = df_demand['index'].tolist()
demand_set = set(demand_nodes)

# 2.3: Build a dictionary for scaled_demand from demand-truck data
#    We only record pairs (i, j) where scaled_demand > 0, to form "validPairs".
scaled_demand = {}
valid_pairs = []
for _, row in df_demand_truck.iterrows():
    i = row['truck_node_index']
    j = row['demand_node_index']
    sd = row['scaled_demand']  # how many burritos truck i can serve to node j
    if sd > 0:
        scaled_demand[(i, j)] = sd
        valid_pairs.append((i, j))

# 2.4: Problem data for profit
burrito_price = df_problem['burrito_price'].iloc[0]       # e.g. 10
ingredient_cost = df_problem['ingredient_cost'].iloc[0]   # e.g. 5
ingredient_cost = 4
profit_per_burrito = burrito_price - ingredient_cost       # e.g. 5

# 2.5: Fixed cost per truck
truck_cost_value = df_problem['truck_cost'].iloc[0]  # e.g. 250
truck_fixed_cost = { t: truck_cost_value for t in truck_locations }

# ---------------------------
# Step 3: Create Gurobi Model
# ---------------------------
m = gp.Model("Burrito_Profit_UsingScaledDemand")

# 3.1: Decision Variables
# z_i: 1 if truck i is open
z = m.addVars(truck_locations, vtype=GRB.BINARY, name="OpenTruck")

# x_{i,j}: 1 if truck i serves node j (which means scaled_demand[(i,j)] burritos)
# Only define x_{i,j} for valid pairs
x = {}
for (i, j) in valid_pairs:
    x[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f"Assign_{i}_{j}")

# 3.2: Constraints

# A) Each demand node must be served exactly once
#    Summation over all trucks i for which (i, j) is a valid pair
for j in demand_nodes:
    # Find which trucks can serve j (scaled_demand>0)
    trucks_that_can_serve_j = [(i, jj) for (i, jj) in valid_pairs if jj == j]
    # If no truck can serve j, the model might be infeasible. 
    if trucks_that_can_serve_j:
        m.addConstr(
            quicksum(x[(i, jj)] for (i, jj) in trucks_that_can_serve_j) == 1,
            name=f"OneTruck_{j}"
        )
    else:
        # Potentially raise an error or handle if no truck can serve j
        print(f"WARNING: No valid truck can serve demand node {j}")

# B) Truck must be open if it serves any demand node
for (i, j) in valid_pairs:
    m.addConstr(x[(i, j)] <= z[i], name=f"Activation_{i}_{j}")

# (Optional) If you do have a capacity limit for each truck, define it:
# For now, let's assume no capacity limit or a big one. 
# If you have capacity[i], do:
# for i in truck_locations:
#     m.addConstr(
#         quicksum(scaled_demand[(i, j)] * x[(i, j)] for (ii, j) in valid_pairs if ii == i) <= capacity[i] * z[i],
#         name=f"Cap_{i}"
#     )

# 3.3: Objective Function
# Maximize total profit = sum( profit_per_burrito * scaled_demand[(i,j)] ) - sum( truck_cost * z[i] )
m.setObjective(
    quicksum( profit_per_burrito * scaled_demand[(i, j)] * x[(i, j)]
              for (i, j) in valid_pairs )
    - quicksum( truck_fixed_cost[i] * z[i] for i in truck_locations ),
    GRB.MAXIMIZE
)

# ---------------------------
# Step 4: Optimize
# ---------------------------
m.optimize()

# ---------------------------
# Step 5: Print Results
# ---------------------------
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal Profit = {m.objVal:.2f}\n")
    chosen_trucks = [i for i in truck_locations if z[i].X > 0.5]
    print("Chosen Truck Locations:", chosen_trucks)

    # For each chosen truck, see which demand nodes it serves
    for i in chosen_trucks:
        assigned_nodes = []
        total_burritos = 0
        for j in demand_nodes:
            if (i, j) in valid_pairs and x[(i, j)].X > 0.5:
                assigned_nodes.append(j)
                total_burritos += scaled_demand[(i, j)]
        if assigned_nodes:
            print(f"  Truck {i} serves: {assigned_nodes}, total scaled burritos = {total_burritos}")
else:
    print("No optimal solution found. Status =", m.status)


Demand Node Columns: ['index', 'name', 'x', 'y', 'demand']
Truck Node Columns: ['index', 'x', 'y']
Demand-Truck Columns: ['demand_node_index', 'truck_node_index', 'distance', 'scaled_demand']
Problem Data Columns: ['burrito_price', 'ingredient_cost', 'truck_cost']
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 405 rows, 395 columns and 1110 nonzeros
Model fingerprint: 0x3e1b4337
Variable types: 0 continuous, 395 integer (395 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 388.0000000
Presolve time: 0.01s
Presolved: 405 rows, 395 columns, 1110 nonzeros
Variable types: 0 continuous, 395 integer (395 binary)

Root relax

In [ ]:
#worse case near break even startgery ingredient cost becomes 9 dollars ? can we still earn profit or break even 

In [10]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum

# ---------------------------
# Step 1: Load Data
# ---------------------------
# df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
# df_truck = pd.read_csv("round1-day2_truck_node_data.csv")
# df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
# df_problem = pd.read_csv("round1-day2_problem_data.csv")

# Load the CSV files
df_demand = pd.read_csv("round1-day2_demand_node_data.csv")
df_truck = pd.read_csv("round1-day2_truck_node_data (1).csv")
df_demand_truck = pd.read_csv("round1-day2_demand_truck_data.csv")
df_problem = pd.read_csv("round1-day2_problem_data.csv")

print("Demand Node Columns:", df_demand.columns.tolist())
print("Truck Node Columns:", df_truck.columns.tolist())
print("Demand-Truck Columns:", df_demand_truck.columns.tolist())
print("Problem Data Columns:", df_problem.columns.tolist())

# ---------------------------
# Step 2: Build Data Structures
# ---------------------------

# 2.1: Truck list
truck_locations = df_truck['index'].tolist()

# 2.2: Demand node list
#    We'll also keep a set for quick membership checks
demand_nodes = df_demand['index'].tolist()
demand_set = set(demand_nodes)

# 2.3: Build a dictionary for scaled_demand from demand-truck data
#    We only record pairs (i, j) where scaled_demand > 0, to form "validPairs".
scaled_demand = {}
valid_pairs = []
for _, row in df_demand_truck.iterrows():
    i = row['truck_node_index']
    j = row['demand_node_index']
    sd = row['scaled_demand']  # how many burritos truck i can serve to node j
    if sd > 0:
        scaled_demand[(i, j)] = sd
        valid_pairs.append((i, j))

# 2.4: Problem data for profit
burrito_price = df_problem['burrito_price'].iloc[0]       # e.g. 10
ingredient_cost = df_problem['ingredient_cost'].iloc[0]   # e.g. 5
ingredient_cost = 9.1
profit_per_burrito = burrito_price - ingredient_cost       # e.g. 5

# 2.5: Fixed cost per truck
truck_cost_value = df_problem['truck_cost'].iloc[0]  # e.g. 250
truck_fixed_cost = { t: truck_cost_value for t in truck_locations }

# ---------------------------
# Step 3: Create Gurobi Model
# ---------------------------
m = gp.Model("Burrito_Profit_UsingScaledDemand")

# 3.1: Decision Variables
# z_i: 1 if truck i is open
z = m.addVars(truck_locations, vtype=GRB.BINARY, name="OpenTruck")

# x_{i,j}: 1 if truck i serves node j (which means scaled_demand[(i,j)] burritos)
# Only define x_{i,j} for valid pairs
x = {}
for (i, j) in valid_pairs:
    x[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f"Assign_{i}_{j}")

# 3.2: Constraints

# A) Each demand node must be served exactly once
#    Summation over all trucks i for which (i, j) is a valid pair
for j in demand_nodes:
    # Find which trucks can serve j (scaled_demand>0)
    trucks_that_can_serve_j = [(i, jj) for (i, jj) in valid_pairs if jj == j]
    # If no truck can serve j, the model might be infeasible. 
    if trucks_that_can_serve_j:
        m.addConstr(
            quicksum(x[(i, jj)] for (i, jj) in trucks_that_can_serve_j) == 1,
            name=f"OneTruck_{j}"
        )
    else:
        # Potentially raise an error or handle if no truck can serve j
        print(f"WARNING: No valid truck can serve demand node {j}")

# B) Truck must be open if it serves any demand node
for (i, j) in valid_pairs:
    m.addConstr(x[(i, j)] <= z[i], name=f"Activation_{i}_{j}")

# (Optional) If you do have a capacity limit for each truck, define it:
# For now, let's assume no capacity limit or a big one. 
# If you have capacity[i], do:
# for i in truck_locations:
#     m.addConstr(
#         quicksum(scaled_demand[(i, j)] * x[(i, j)] for (ii, j) in valid_pairs if ii == i) <= capacity[i] * z[i],
#         name=f"Cap_{i}"
#     )

# 3.3: Objective Function
# Maximize total profit = sum( profit_per_burrito * scaled_demand[(i,j)] ) - sum( truck_cost * z[i] )
m.setObjective(
    quicksum( profit_per_burrito * scaled_demand[(i, j)] * x[(i, j)]
              for (i, j) in valid_pairs )
    - quicksum( truck_fixed_cost[i] * z[i] for i in truck_locations ),
    GRB.MAXIMIZE
)

# ---------------------------
# Step 4: Optimize
# ---------------------------
m.optimize()

# ---------------------------
# Step 5: Print Results
# ---------------------------
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal Profit = {m.objVal:.2f}\n")
    chosen_trucks = [i for i in truck_locations if z[i].X > 0.5]
    print("Chosen Truck Locations:", chosen_trucks)

    # For each chosen truck, see which demand nodes it serves
    for i in chosen_trucks:
        assigned_nodes = []
        total_burritos = 0
        for j in demand_nodes:
            if (i, j) in valid_pairs and x[(i, j)].X > 0.5:
                assigned_nodes.append(j)
                total_burritos += scaled_demand[(i, j)]
        if assigned_nodes:
            print(f"  Truck {i} serves: {assigned_nodes}, total scaled burritos = {total_burritos}")
else:
    print("No optimal solution found. Status =", m.status)


Demand Node Columns: ['index', 'name', 'x', 'y', 'demand']
Truck Node Columns: ['index', 'x', 'y']
Demand-Truck Columns: ['demand_node_index', 'truck_node_index', 'distance', 'scaled_demand']
Problem Data Columns: ['burrito_price', 'ingredient_cost', 'truck_cost']
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 405 rows, 395 columns and 1110 nonzeros
Model fingerprint: 0xe95fb50c
Variable types: 0 continuous, 395 integer (395 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -3554.300000
Presolve time: 0.01s
Presolved: 405 rows, 395 columns, 1110 nonzeros
Variable types: 0 continuous, 395 integer (395 binary)

Root rela